<a href="https://colab.research.google.com/github/Parikshit07111996/School-Of-AI/blob/master/Assignment_File_Session_5_(5)_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Target
1. Adding Regularization , droput
2. Adding Global Average Pooling to get rid of Big kernel (7 x 7)

# Results
1. Parameters : 9.9k
2. Best Training Accuracy : 98.16
3. Best Test Accuracy : 98.9

#Analysis
1. Unable to achieve the 99.4% accuracy
 

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

#Data Transform



In [0]:
# Train Phase Transformation
train_transforms = transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,),(0.3081,))
                                       ])


# Test Phase Transformation
test_transforms = transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,),(0.3081,))
                                       ])

# Dataset and Creating Train/Test Split

In [3]:
train = datasets.MNIST('./data',train=True, download=True, transform= train_transforms)
test = datasets.MNIST('./data',train=True, download=True, transform= test_transforms)

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


#Dataloader Arguments & Test/Train Dataloaders

In [0]:
SEED = 1

cuda = torch.cuda.is_available()

torch.manual_seed(SEED)

if cuda:
  torch.cuda.manual_seed(SEED)

dataloader_args = dict(shuffle=True, batch_size= 128, num_workers=4, pin_memory=True)

train_loader = torch.utils.data.DataLoader(train, **dataloader_args)

test_loader = torch.utils.data.DataLoader(test, **dataloader_args)


#The Model

In [0]:
dropout_value = 0.1
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #Input Block
        self.convblock1 = nn.Sequential(
          nn.Conv2d(1, 10, 3, 1,False),
          nn.ReLU(),
          nn.BatchNorm2d(10),
          nn.Dropout(dropout_value)               
         ) # Output size = 26
        
        #Convolution Block 1
        self.convblock2 = nn.Sequential(
        nn.Conv2d(10, 10, 3,1,False),
        nn.ReLU(),
        nn.BatchNorm2d(10),
        nn.Dropout(dropout_value)
        ) # Output size = 24

        self.convblock3 = nn.Sequential(
        nn.Conv2d(10, 10, 3,1,False),
        nn.ReLU(),
        nn.BatchNorm2d(10),
        nn.Dropout(dropout_value)    
        ) # Output size = 22

        #Transition Block 1

        self.pool1 = nn.MaxPool2d(2, 2) # Outout_size = 11

        self.convblock4 = nn.Sequential(
            nn.Conv2d(10, 10, 1),
            nn.ReLU(),
            nn.BatchNorm2d(10),
            nn.Dropout(dropout_value)
        ) # Outout_size = 11

        # Convolution Block 2       
        self.convblock5 = nn.Sequential(
        nn.Conv2d(10, 10, 3),
        nn.ReLU(),
        nn.BatchNorm2d(10),
        nn.Dropout(dropout_value)    
        ) # output size 9

        self.convblock6 = nn.Sequential(
        nn.Conv2d(10, 20, 3),
        nn.ReLU(),
        nn.BatchNorm2d(20),
        nn.Dropout(dropout_value)    
        ) # output size 7

        # Output Block 
        self.convblock7 = nn.Sequential(
        nn.Conv2d(20, 10, 1),
        nn.BatchNorm2d(10),
        nn.Dropout(dropout_value)
        )  # output size 7

        self.gap = nn.Sequential(
        nn.AvgPool2d(5),
        )  # output size 1

        self.convblock8 = nn.Sequential(
        nn.Conv2d(10, 10, 1)
        )  # output size 7
        
        self.dropout = nn.Dropout(dropout_value)
        
        

    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.dropout(x)
        x = self.pool1(x)
        x = self.convblock4(x)
        x = self.convblock5(x)
        x = self.convblock6(x)
        x = self.dropout(x)
        x = self.convblock7(x)
        x = self.gap(x)
        x = self.convblock8(x)
        
        x = x.view(-1, 10)
        return F.log_softmax(x)

#Model Parameters

In [26]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 26, 26]             100
              ReLU-2           [-1, 10, 26, 26]               0
       BatchNorm2d-3           [-1, 10, 26, 26]              20
           Dropout-4           [-1, 10, 26, 26]               0
            Conv2d-5           [-1, 10, 24, 24]             910
              ReLU-6           [-1, 10, 24, 24]               0
       BatchNorm2d-7           [-1, 10, 24, 24]              20
           Dropout-8           [-1, 10, 24, 24]               0
            Conv2d-9           [-1, 10, 22, 22]             910
             ReLU-10           [-1, 10, 22, 22]               0
      BatchNorm2d-11           [-1, 10, 22, 22]              20
          Dropout-12           [-1, 10, 22, 22]               0
          Dropout-13           [-1, 10, 22, 22]               0
        MaxPool2d-14           [-1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:88: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


#Training and Testing

In [0]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []



def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    processed = 0
    for batch_idx, (data, target) in enumerate(pbar):
      #get samples
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        y_pred = model(data)
        loss = F.nll_loss(y_pred, target)
        loss.backward()
        optimizer.step()

        pred = y_pred.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)
        pbar.set_description(desc= f'loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.3f}')
        train_acc.append(100*correct/processed)


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

#Train & Test Our Model

In [28]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
EPOCHS = 15

for epoch in range(EPOCHS):
  print("EPOCH:", epoch)
  train(model, device, train_loader, optimizer, epoch)
  test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]

EPOCH: 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:88: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
loss=0.15855072438716888 Batch_id=468 Accuracy=78.437: 100%|██████████| 469/469 [00:11<00:00, 42.45it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.1752, Accuracy: 57145/60000 (95.242%)

EPOCH: 1


loss=0.12007518857717514 Batch_id=468 Accuracy=95.165: 100%|██████████| 469/469 [00:10<00:00, 43.03it/s]
  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0399, Accuracy: 59271/60000 (98.785%)

EPOCH: 13


loss=0.07245953381061554 Batch_id=468 Accuracy=98.127: 100%|██████████| 469/469 [00:10<00:00, 42.67it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0368, Accuracy: 59310/60000 (98.850%)

EPOCH: 14


loss=0.015785658732056618 Batch_id=468 Accuracy=98.162: 100%|██████████| 469/469 [00:10<00:00, 43.22it/s]



Test set: Average loss: 0.0358, Accuracy: 59346/60000 (98.910%)

